In [ ]:
%load_ext cython

In [ ]:
import asyncio

In [ ]:
def run_async(coro):
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(coro)
    return result

In [ ]:
from array import array

async def int_fizz_buzz(stream, callback=None):
    rest = b''
    avalues = array('i')
    async for data in stream:
        if rest:
            data = rest + data
            rest = b''

        remainder = len(data) % 4
        if remainder:
            rest, data = data[-remainder:], data[:-remainder]
        avalues.frombytes(data)

        new_values = []
        for value in avalues:
            new_values.append(
                'fizzbuzz' if value % 15 == 0
                else 'fizz' if value % 3 == 0
                else 'buzz' if value % 5 == 0
                else value
            )
        if avalues and callback is not None:
            callback(new_values)
        del avalues[:]


In [ ]:
%%cython -3 -a

cdef int chars_to_int(unsigned char* data):
    return (<int*>data)[0]  # TODO: byte order mapping :)

async def cint_fizz_buzz(stream, callback=None):
    cdef bytes data, rest = b''
    cdef Py_ssize_t i, remainder
    cdef unsigned char* c_data
    cdef int value
    assert sizeof(int) == 4

    async for data in stream:
        if rest:
            data = rest + data
            rest = b''

        c_data = data
        new_values = []
        for i in range(0, len(data) - 3, 4):
            value = chars_to_int(c_data + i)
            if value < 1:
                continue
            new_values.append(
                'fizzbuzz' if value % 15 == 0
                else 'fizz' if value % 3 == 0
                else 'buzz' if value % 5 == 0
                else value
            )
        if new_values and callback is not None:
            callback(new_values)

        remainder = len(data) % 4
        if remainder:
            rest = data[-remainder:]


In [ ]:
def as_chunks(data, chunk_size=1024):
    for pos in range(0, len(data), chunk_size):
        yield data[pos: pos+chunk_size]

class DataIter:
    def __init__(self, chunks):
        self._data = iter(chunks)
    def __aiter__(self):  # NOTE: used to be "async def" in Py3.5.0, changed in Python 3.5.1 / Cython 0.24.1
        return self
    async def __anext__(self):
        try:
            return next(self._data)
        except StopIteration:
            raise StopAsyncIteration

In [ ]:
expected_18 = [1, 2, 'fizz', 4, 'buzz', 'fizz', 7, 8, 'fizz', 'buzz', 11, 'fizz', 13, 14, 'fizzbuzz', 16, 17, 'fizz']

from array import array
data = array('i', range(1, 19)).tobytes()
chunks_of_4 = list(as_chunks(data, chunk_size=4))
chunks_of_20 = list(as_chunks(data, chunk_size=20))
chunks_of_23 = list(as_chunks(data, chunk_size=23))

In [ ]:
result = []
run_async(cint_fizz_buzz(DataIter(chunks_of_4[:6]), callback=result.extend))
print(result)

In [ ]:
result = []
run_async(cint_fizz_buzz(DataIter(chunks_of_20), callback=result.extend))
print(result)

In [ ]:
result = []
run_async(cint_fizz_buzz(DataIter(chunks_of_23), callback=result.extend))
print(result)

In [ ]:
%%timeit chunks = chunks_of_20 * 1000
run_async(cint_fizz_buzz(DataIter(chunks), callback=id))  # Cython

In [ ]:
%%timeit chunks = chunks_of_20 * 1000
run_async(int_fizz_buzz(DataIter(chunks), callback=id)) # Python

In [ ]:
%%timeit chunks = chunks_of_23 * 1000
run_async(cint_fizz_buzz(DataIter(chunks), callback=id))  # Cython

In [ ]:
%%timeit chunks = chunks_of_23 * 1000
run_async(int_fizz_buzz(DataIter(chunks), callback=id))  # Python